In [6]:
import dgl
import math
import torch
import numpy as np
from os import path
from pathlib import Path
from dgl.data import DGLDataset
from ToyDGLDataset import ToyDGLDataset, LoadToyDGLDataset

In [7]:
datasetName = 'ToyDataset01'
datasetDir = path.join('/home/andrew/GNN_Sandbox/GraphDatasets', datasetName)
dataset = ToyDGLDataset(datasetName, datasetDir)

Done loading data from cached files.


In [8]:
dataset.printProperties()

Num Graph classes: 2
Graph classes: [0, 1]
Number of graphs: 20
Number of all nodes in all graphs: 982
Number of all edges in all graphs: 55454
Dim node features: 5
Node feature keys: ['P_t', 'Eta', 'Phi', 'Mass', 'Type']
Dim edge features: 3
Edge feature keys: ['DeltaPhi', 'DeltaEta', 'RapiditySquared']


In [ ]:
print(len(dataset))
graph, label = dataset[0]
print(graph)
print(f'Label: {label}')

In [ ]:
dataset.printNumbers()
print('DeltaPhi' in graph.ndata.keys())

In [ ]:
from Utils import GetNodeFeatureVec, GetEdgeFeatureVec

print(GetNodeFeatureVec(graph))
print(GetEdgeFeatureVec(graph))
print(graph.ndata.keys())
print(len(graph.ndata.keys()))


In [ ]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
splitIndices = dataset.get_split_indices()

train_sampler = SubsetRandomSampler(splitIndices['train'])
test_sampler = SubsetRandomSampler(splitIndices['test'])

train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=32, drop_last=False)
test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=32, drop_last=False)

In [ ]:
it = iter(train_dataloader)
batch = next(it)
print(batch)

In [ ]:
batched_graph, labels = batch
print('Number of nodes for each graph element in the batch:', batched_graph.batch_num_nodes())
print('Number of edges for each graph element in the batch:', batched_graph.batch_num_edges())
print(labels)
# Recover the original graph elements from the minibatch
graphs = dgl.unbatch(batched_graph)
print('The original graphs in the minibatch:')
print(graphs)

In [ ]:
from dgl.nn import GraphConv
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        return dgl.mean_nodes(g, 'h')

In [ ]:
# Create the model with given dimensions
model = GCN(dataset.dim_nfeats, 16, dataset.num_graph_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(50):
    for batched_graph, labels in train_dataloader:
        nodeFeatVec = GetNodeFeatureVec(batched_graph).float()
        pred = model(batched_graph, nodeFeatVec)
        loss = F.cross_entropy(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #if epoch % 5 == 0:
    print(f'Epoch: {epoch}, Loss: {loss:.3f}')

num_correct = 0
num_tests = 0
for batched_graph, labels in test_dataloader:
    nodeFeatVec = GetNodeFeatureVec(batched_graph).float()
    pred = model(batched_graph, nodeFeatVec)
    num_correct += (pred.argmax(1) == labels).sum().item()
    num_tests += len(labels)

print('Test accuracy:', num_correct / num_tests)

In [ ]:
import torch
a = torch.ones(1)
b = torch.ones(3)
c = torch.randn(5)
print(b)
print(c)
d = torch.cat((b, c))
e = torch.hstack((b, c))
print(d)
print(e)
f = torch.cat((b,c,a))
print(f)